In [3]:
from keras.callbacks import EarlyStopping
from pmdarima.utils import diff_inv
import statsmodels.api as sm
import statsmodels
import random
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.models import Sequential,load_model,Model
from statsmodels.tsa.arima_process import arma_generate_sample
from tensorflow.keras.callbacks import ModelCheckpoint
from collections import defaultdict,Counter
from keras.callbacks import EarlyStopping

In [4]:
def to_put(array,timesteps):
    array=array[:,:timesteps]
    array=array.reshape((array.shape[0],array.shape[1],1))
    return array

In [11]:
fdin=pd.read_csv('c:/data/dataset/fdin.csv',header=None).to_numpy()
fdoutb=pd.read_csv('c:/data/dataset/fdoutb.csv',header=None).to_numpy()
fdoutm=pd.read_csv('c:/data/dataset/fdoutm.csv',header=None).to_numpy()
#
trin=fdin[:100000]
valin=fdin[100000:130000]
tein=fdin[130000:]
#
troutb=fdoutb[:100000]
valoutb=fdoutb[100000:130000]
teoutb=fdoutb[130000:]
#
troutm=fdoutm[:100000]
valoutm=fdoutm[100000:130000]
teoutm=fdoutm[130000:]

In [ ]:
## training binary classifier for statistical hypothesis test
dbL=Sequential([
    LSTM(30,input_shape=(100,1)),
    Dense(2,activation='softmax')
])
dbL.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
early_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
checkpoint=ModelCheckpoint('c:/data/model_weights/dbL.hdf5',monitor='val_loss',verbose=0,mode='min',save_best_only=True)
dbL_history=dbL.fit(to_put(trin,100),troutb,
                         validation_data=(to_put(valin,100),valoutb),batch_size=1000,epochs=200,
                         callbacks=[checkpoint,early_stopping],verbose=0)

In [ ]:
# training 3-class classifier for predicting number of unit roots 
dmL=Sequential([
    LSTM(30,input_shape=(100,1)),
    Dense(3,activation='softmax')
])
dmL.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
early_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
checkpoint=ModelCheckpoint('c:/data/model_weights/dmL.hdf5',monitor='val_loss',verbose=0,mode='min',save_best_only=True)
dmL_history=dmL.fit(to_put(trin,100),troutm,
                         validation_data=(to_put(valin,100),valoutm),batch_size=1000,epochs=200,
                         callbacks=[checkpoint,early_stopping],verbose=0)